In [1]:
import os
from dotenv import load_dotenv
import dask.dataframe as dd
from src.features.csv_to_parquet import ITEM_DTYPES

load_dotenv()


True

In [2]:
ddf = dd.read_csv(
    "s3://food-delivery-crawler/data-interim/processed/*/store/**/*-menu.csv",
    storage_options={"client_kwargs": {"endpoint_url": os.environ["S3_ENDPOINT"]}},
    dtype=ITEM_DTYPES,
    include_path_column="filePath",
)


In [3]:
relevant_columns = [
    "name",
    "id",
    "description",
    "categoryId",
    "price_formatted",
    "filePath",
    "menuRow",
]
ddf = ddf.rename(columns={"Unnamed: 0": "menuRow"})[relevant_columns]
store_ddf = (
    ddf.filePath.str.removesuffix("-menu.csv")
    .str.split("/", expand=True, n=9)[[5, 6, 9]]
    .rename(columns={5: "city", 6: "zone", 9: "store"})
)

output_ddf = ddf.assign(city=store_ddf.city, zone=store_ddf.zone, store=store_ddf.store)
output_ddf = output_ddf.repartition(partition_size="1MB")


In [4]:
output_ddf.head()

,name,id,description,categoryId,price_formatted,filePath,menuRow,city,zone,store
0,Panino kebab e bibita,1054925545,NaN,138580330,€7.00,food-delivery-crawler/data-interim/processed/d...,0,bergamo,alzano-lombardo,happy-pizza-bergamo
1,"Panino kebab, bibita e patatine",1054925614,NaN,138580330,€8.00,food-delivery-crawler/data-interim/processed/d...,1,bergamo,alzano-lombardo,happy-pizza-bergamo
2,Piadina kebab e bibita,1054925563,NaN,138580330,€7.50,food-delivery-crawler/data-interim/processed/d...,2,bergamo,alzano-lombardo,happy-pizza-bergamo
3,"Piadina kebab, bibita e patatine",1054925557,NaN,138580330,€8.50,food-delivery-crawler/data-interim/processed/d...,3,bergamo,alzano-lombardo,happy-pizza-bergamo
4,Piatto kebab e bibita,1054925682,NaN,138580330,€9.50,food-delivery-crawler/data-interim/processed/d...,4,bergamo,alzano-lombardo,happy-pizza-bergamo
